In [1]:
import numpy as np
import pandas as pd
from src import bayesian_3pt_percentage_with_credible_interval
from sqlalchemy import create_engine
from utils import mean_excluding_outliers, find_players_similar_3PCT

In [2]:
engine = create_engine("sqlite:///Data/player_database.db")
input_data = pd.read_sql("BAYESIAN_3PT_INPUT_2023_24", engine)
common_player_info = pd.read_sql("COMMON_PLAYER_INFO", engine)
player_career_stats = pd.read_sql("PLAYER_CAREER_STATS", engine)
free_agents = pd.read_sql("FREE_AGENTS_2023_24", engine)
salary_data = pd.read_sql("PLAYER_SALARY_DATA_2023_24", engine)
input_data = input_data.dropna()

In [3]:
print(free_agents.columns)

Index(['Name'], dtype='object')


In [4]:
salary_data['2024-25'] = salary_data['2024-25'].str.replace('$', '', regex=False)
salary_data['2024-25'] = salary_data['2024-25'].str.replace(',', '', regex=False)
salary_data['2024-25'] = salary_data["2024-25"].astype(float)

In [5]:
players = []
player_names = []
bayesian_three_perc = []
ordinary_three_perc = []
historical_three_perc = []
lower_bounds = []
upper_bounds = []
avg_3pa_per_game = []
bayesian_percentage_salary = []
season_percentage_salary = []

In [6]:
for index, row in input_data.iterrows():
    num_seasons = len(np.unique(player_career_stats[player_career_stats["PLAYER_ID"] == row["PLAYER_ID"]]["SEASON_ID"]))
    player_3pa = np.sum(player_career_stats[player_career_stats["PLAYER_ID"] == row["PLAYER_ID"]]["FG3A"])
    player_avg_3pa_season = float(player_3pa)/float(num_seasons)
    
    
    if row["SEASON_3PA"] > 0 and player_avg_3pa_season > 250:
        name_matches = common_player_info[common_player_info["PERSON_ID"] == row["PLAYER_ID"]]["DISPLAY_FIRST_LAST"].tolist()
        if len(name_matches) == 1:
            players.append(int(row["PLAYER_ID"]))
            player_names.append(name_matches[0])
            
            perc, lower_bound, upper_bound = bayesian_3pt_percentage_with_credible_interval(float(row["HISTORICAL_3P%"]), row["SEASON_3PA"], row["SEASON_3PM"])
            bayesian_three_perc.append(perc)
            lower_bounds.append(lower_bound)
            upper_bounds.append(upper_bound)
            ordinary_three_perc.append(row["SEASON_3PM"]/row["SEASON_3PA"])
            historical_three_perc.append(row["HISTORICAL_3P%"])
            avg_3pa_per_game.append(float(player_3pa) / (82*num_seasons))

In [7]:
all_players = {"PLAYER_ID": players, "PLAYER": player_names, "BAYESIAN_3P%": bayesian_three_perc, "LOWER_BOUND": lower_bounds, "UPPER_BOUND": upper_bounds, "SEASON_3P%": ordinary_three_perc, "HISTORICAL_3P%": historical_three_perc, "AVG_3PA_PER_GAME": avg_3pa_per_game}
all_players = pd.DataFrame(all_players)

In [8]:
#Get comparable salaries
bayesian_comparable_salary = []
season_comparable_salary = []
for index, row in all_players.iterrows():
    bayesian_comparable_players = find_players_similar_3PCT(row["BAYESIAN_3P%"], all_players[["PLAYER", "BAYESIAN_3P%"]])
    season_comparable_players = find_players_similar_3PCT(row["SEASON_3P%"], all_players[["PLAYER", "SEASON_3P%"]])
    
    b_mean = mean_excluding_outliers(salary_data[salary_data["Player"].isin(bayesian_comparable_players)]["2024-25"])
    s_mean = mean_excluding_outliers(salary_data[salary_data["Player"].isin(season_comparable_players)]["2024-25"])
    bayesian_comparable_salary.append(b_mean)
    season_comparable_salary.append(s_mean)

all_players["B_COMP_SALARY"] = bayesian_comparable_salary
all_players["S_COMP_SALARY"] = season_comparable_salary

In [9]:
df = all_players[all_players["PLAYER"].isin(free_agents["Name"])]

In [10]:
print(df)

    PLAYER_ID             PLAYER  BAYESIAN_3P%  LOWER_BOUND  UPPER_BOUND  \
6      201587      Nicolas Batum      0.364918     0.169169     0.560561   
15    1628970      Miles Bridges      0.345916     0.150150     0.541542   
20     201144        Mike Conley      0.382334     0.186186     0.578579   
21     203496   Robert Covington      0.361138     0.165165     0.557558   
22     203109        Jae Crowder      0.342729     0.147147     0.538539   
25     203915  Spencer Dinwiddie      0.339516     0.144144     0.535536   
32     201568   Danilo Gallinari      0.379905     0.184184     0.575576   
37     201980        Danny Green      0.400063     0.204204     0.595596   
43     202699      Tobias Harris      0.373031     0.177177     0.568569   
48     203200     Justin Holiday      0.359673     0.164164     0.555556   
63     200768         Kyle Lowry      0.365344     0.169169     0.561562   
65     202083    Wesley Matthews      0.375606     0.180180     0.571572   
83    162775

In [11]:
df.to_sql("BAYESIAN_3PT_OUTPUT_2023_24", con=engine, if_exists="replace",index=False)

14

In [12]:
df = df.dropna()

In [13]:
df.to_csv("free_agent_3pt_output_2023_24.csv")

In [14]:
print(df)

    PLAYER_ID             PLAYER  BAYESIAN_3P%  LOWER_BOUND  UPPER_BOUND  \
6      201587      Nicolas Batum      0.364918     0.169169     0.560561   
15    1628970      Miles Bridges      0.345916     0.150150     0.541542   
20     201144        Mike Conley      0.382334     0.186186     0.578579   
21     203496   Robert Covington      0.361138     0.165165     0.557558   
22     203109        Jae Crowder      0.342729     0.147147     0.538539   
25     203915  Spencer Dinwiddie      0.339516     0.144144     0.535536   
32     201568   Danilo Gallinari      0.379905     0.184184     0.575576   
43     202699      Tobias Harris      0.373031     0.177177     0.568569   
48     203200     Justin Holiday      0.359673     0.164164     0.555556   
63     200768         Kyle Lowry      0.365344     0.169169     0.561562   
65     202083    Wesley Matthews      0.375606     0.180180     0.571572   
83    1627752     Taurean Prince      0.373932     0.178178     0.569570   
89     20396